In [ ]:
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from dags.utils import MINIO_ACCESS_KEY, MINIO_SECRET_KEY, MINIO_HOST

# Dùng Delta 2.4.0 (có trên Maven, ổn định cho Spark 4.0)
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages "
    "org.apache.hadoop:hadoop-aws:3.3.4,"
    "com.amazonaws:aws-java-sdk-bundle:1.12.767,"
    "io.delta:delta-core_2.12:2.4.0 "
    "pyspark-shell"
)

def get_spark_session():
    builder = (
        SparkSession.builder
        .appName("DeltaLakeReader")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO configs
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        .config("spark.hadoop.fs.s3a.endpoint", f"http://{MINIO_HOST}")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    )
    return configure_spark_with_delta_pip(builder).getOrCreate()

# Khởi tạo Spark
spark = get_spark_session()

# Đọc Delta từ MinIO
df = spark.read.format("delta").load("s3a://datn/raw/careerviet/recruit")

# Show 5 dòng đầu
df.show(5)


your 131072x1 screen size is bogus. expect trouble
25/07/27 10:48:44 WARN Utils: Your hostname, DESKTOP-HHC5U09 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/27 10:48:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/root/.pyenv/versions/3.10.15/envs/sales-smart/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-92de493f-9042-4fdd-928e-4c89ba758756;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.767 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (2046ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.d

In [1]:
from deltalake import DeltaTable

# Kết nối đến MinIO
dt = DeltaTable(
    "s3://datn/raw/careerviet/recruit",
    storage_options={
        "AWS_ACCESS_KEY_ID": "minioadmin",
        "AWS_SECRET_ACCESS_KEY": "minioadmin",
        "AWS_ENDPOINT_URL": "http://localhost:9000",
        "AWS_REGION": "us-east-1",
        "AWS_ALLOW_HTTP": "true",
        "AWS_S3_FORCE_PATH_STYLE": "true"
    }
)

# Đọc thành Pandas và chỉ lấy cột job_id
df = dt.to_pandas(columns=["job_id"])

# Chuyển sang list
job_ids = df["job_id"].tolist()



OSError: Generic S3 error
          [31m↳[0m Error performing GET http://localhost:9000/datn/raw/careerviet/recruit/_delta_log/_last_checkpoint in 1.286741ms - HTTP error
           [31m↳[0m error sending request
